# Partie 1 : UPGMA

##### A Quick Implementation of UPGMA (Unweighted Pair Group Method with Arithmetic Mean)



In [1]:
def get_lowest(matrix):
    min_cell = float("inf")
    x, y = -1, -1
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] < min_cell:
                min_cell = matrix[i][j]
                x, y = i, j
    return x, y

In [2]:
def concate_labels(lab, a, b):
    if b < a:
        a, b = b, a
    lab[a] = "(" + lab[a] + "," + lab[b] + ")"
    del lab[b]

In [3]:
def join_table(table, a, b):
    if b < a:
        a, b = b, a

    row = []
    for i in range(0, a):
        row.append((table[a][i] + table[b][i])/2)
    table[a] = row
    
    for i in range(a+1, b):
        table[i][a] = (table[i][a]+table[b][i])/2
    
    for i in range(b+1, len(table)):
        table[i][a] = (table[i][a]+table[i][b])/2
        del table[i][b]
    
    del table[b]

In [4]:
def UPGMA(matrix, lab):
    while len(lab) > 1:
        x, y = get_lowest(matrix)
        join_table(matrix, x, y)
        concate_labels(lab, x, y)
    return lab[0]

In [5]:
def alpha_labels(start, end):
    labels = []
    for i in range(ord(start), ord(end)+1):
        labels.append(chr(i))
    return labels

##### Initialzing data and applic the UPGMA Algo

In [6]:
M_labels = alpha_labels("A", "D")   #A through D
M = [
    [],                       #A
    [17],                      #B
    [21, 12],                   #C
    [27, 18, 14],                #D
    ]

In [7]:
UPGMA(M, M_labels)

'(A,((B,C),D))'

# Partie 2 : alignements simples et multiples

##### 2.1. Read data from fasta file

In [8]:
prot_dict = {}
with open("Annexin1-2-6-2r.fasta", "r") as fasta_file:
    prot_id = ""
    for line in fasta_file:
        if line.startswith(">"):
            prot_id = line[1:].split()[0]
            prot_dict[prot_id] = ""
        else:
            prot_dict[prot_id] += line.strip()
    for id in prot_dict:
        print(id)
        print(prot_dict[id][:30])

sp|P04083|ANXA1_HUMAN
MAMVSEFLKQAWFIENEEQEYVQTVKSSKG
sp|P07355|ANXA2_HUMAN
MSTVHEILCKLSLEGDHSTPPSAYGSVKAY
sp|P08133|ANXA6_HUMAN
MAKPAQGAKYRGSIHDFPGFDPNQDAEALY
sp|Q3ZCQ2|AX2R_HUMAN
MEQHFLGCVKRAWDSAEVAPEPQPPPIVSS


In [9]:
p1 = prot_dict["sp|P04083|ANXA1_HUMAN"]
p2 = prot_dict["sp|P07355|ANXA2_HUMAN"]
p6 = prot_dict["sp|P08133|ANXA6_HUMAN"]
p2r = prot_dict["sp|Q3ZCQ2|AX2R_HUMAN"]

#### 2.2. BLOSUM-50

In [10]:
from Bio.SubsMat import MatrixInfo as matlist
from Bio import pairwise2, AlignIO, Phylo

C:\Anaconda3\lib\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [ ]:
# print(help(Bio))
# print(help(matlist))

In [11]:
def alignement_simple(matrix, p1_, p2_):
    insertion = -10
    deletion = -3
    al = pairwise2.align.globalds(p1_, p2_, matrix, insertion, deletion)[0]
    return al

In [12]:
matrix50=matlist.blosum50
# print("BLOSUM-50 : ", matrix)
hp1, hp2, score, begin, end = alignement_simple(matrix50, p1, p2)
print("\nRESULT SCORE :  ", score)
print("\n------------------------- Clustal Formating :  -------------------------")
print("PROTINE 01 = \n",hp1.format('clustal'))
print("PROTINE 02 = \n",hp2.format('clustal'))



RESULT SCORE :   1086.0

------------------------- Clustal Formating :  -------------------------
PROTINE 01 = 
 MAMVSEFLKQAWFIENEEQEYVQTVKSSKGGPGSAVSPYPTFNPSSDVAALHKAIMVKGVDEATIIDILTKRNNAQRQQIKAAYLQETGKPLDETLKKALTGHLEEVVLALLKTPAQFDADELRAAMKGLGTDEDTLIEILASRTNKEIRDINRVYREELKRDLAKDITSDTSGDFRNALLSLAKGDRSEDFGV-NEDLADSDARALYEAGERRKGTDVNVFNTILTTRSYPQLRRVFQKYTKYSKHDMNKVLDLELKGDIEKCLTAIVKCATSKPAFFAEKLHQAMKGVGTRHKALIRIMVSRSEIDMNDIKAFYQKMYGISLCQAILDETKGDYEKILVALCGGN-
PROTINE 02 = 
 MSTVHEILCK-LSLEGDH----STPPSAYG----SVKAYTNFDAERDALNIETAIKTKGVDEVTIVNILTNRSNAQRQDIAFAYQRRTKKELASALKSALSGHLETVILGLLKTPAQYDASELKASMKGLGTDEDSLIEIICSRTNQELQEINRVYKEMYKTDLEKDIISDTSGDFRKLMVALAKGRRAEDGSVIDYELIDQDARDLYDAGVKRKGTDVPKWISIMTERSVPHLQKVFDRYKSYSPYDMLESIRKEVKGDLENAFLNLVQCIQNKPLYFADRLYDSMKGKGTRDKVLIRIMVSRSEVDMLKIRSEFKRKYGKSLYYYIQQDTKGDYQKALLYLCGGDD


#### 2.3. BLOSUM-80

In [13]:
matrix80=matlist.blosum80
hp1, hp2, score, begin, end = alignement_simple(matrix80, p1, p2)
print("\nRESULT SCORE :  ", score)


RESULT SCORE :   843.0


#### 2.4. Calcule Taux

In [14]:
def get_taux(x, y, longeur):
    inc=0
    for i, j in zip(x, y):
        if i==j:
            inc=inc+1
    return inc/longeur

In [15]:
print("Taux : ", get_taux(hp1, hp2, len(p1)))

Taux :  0.5202312138728323


#### 2.5. Effectuer l’alignement multiple

In [16]:
# Open and initiate the Distance Calculator using the Identity model 
from Bio.Phylo.TreeConstruction import DistanceCalculator 
calculator = DistanceCalculator('identity')

In [17]:
# Open the alignment file as a MultipleSeqAlignment object 
with open("muscle.aln","r") as aln: 
    alignment = AlignIO.read(aln,"clustal")
print(type(alignment))

<class 'Bio.Align.MultipleSeqAlignment'>


#### 2.6. Calculer la séquence consensus.

In [18]:
from Bio.Align import AlignInfo
summary_align = AlignInfo.SummaryInfo(alignment)

In [ ]:
# Calculating a quick consensus sequence
consensus = summary_align.dumb_consensus()
print("Consensus : ", consensus)

#### 2.7. Charger et afficher l’arbre phylogénétique

In [ ]:
# Write the Distance Matrix 
distance_matrix = calculator.get_distance(alignment)
print(distance_matrix)

In [ ]:
# Open and initiate the Tree Constructor 
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
constructor = DistanceTreeConstructor(calculator)

In [ ]:
# Build the tree 
anexine_tree = constructor.build_tree(alignment)
anexine_tree.rooted = True
print(anexine_tree)

In [ ]:
# Save the tree to a new file 
Phylo.write(anexine_tree, "anexineTree.xml", "phyloxml")

In [ ]:
# Import matplotlib and create a basic tree 
import matplotlib
import matplotlib.pyplot as plt
# fig = Phylo.draw(anexine_tree)
fig = plt.figure(figsize=(13, 5), dpi=100) # create figure & set the size 
matplotlib.rc('font', size=12)              # fontsize of the leaf and node labels 
matplotlib.rc('xtick', labelsize=10)       # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=10)       # fontsize of the tick labels
#turtle_tree.ladderize()
axes = fig.add_subplot(1, 1, 1)
Phylo.draw(anexine_tree, axes=axes)
fig.savefig("Anexine Protine Famile")